In [54]:
import pandas as pd
import numpy as np
import requests
import datetime
import json
import plotly.graph_objs as go
import plotly.plotly as py
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode() 
import Quandl

py.sign_in('enrico.tesio','ttv7w7heht')

# Get data FROM U.S. Energy Information Administration using the below API key
api_key = '508B0FE56B9D2507A2CE82A700EE86D0'
req = 'http://api.eia.gov/series/?api_key={}&series_id=NG.N3010{}2.M'.format(api_key,'US')
r_us = requests.get(req)
data_us = json.loads(r_us.text)
dates = []
volumes = []
# set index using the total U.S. data
for element in data_us['series'][0]['data']:
    dates.append(datetime.datetime.strptime(element[0], '%Y%m').date())
    volumes.append(float(element[1]))    
df = pd.DataFrame(index=dates, columns=['US Consumption'])
df['US Consumption'] = volumes
df.sort_index(ascending=True, inplace=True)
henry_hub_m1 = Quandl.get('FRED/GASPRICE')

conversion = 0.9756 # MMBTU to MMcft
df = pd.merge(df, henry_hub_m1, how='outer', left_index=True, right_index=True).dropna(how='any')*conversion
df.columns = ['US Consumption', 'HH M1']
df = df.pct_change()

trace_1 = go.Scatter(x=df.index, y=df['US Consumption'], name='U.S. Consumption', yaxis='y1')
trace_2 = go.Scatter(x=df.index, y=df['HH M1'], name='Henry Hub Month-Ahead', yaxis='y2')
data = [trace_1, trace_2]
layout = dict(title='U.S. consumption and Henry Hub price (1990 - Today)',
             xaxis=dict(title='Time'),
             yaxis=dict(title='US gas consumption (MMcft)'),
             yaxis2=dict(title='Henry Hub Month-Ahead price ($/MMcft)', side='right',
                        titlefont=dict(color='rgb(148, 103, 189)'),tickfont=dict(color='rgb(148, 103, 189)'),
                         overlaying='y'),
             legend=dict(x=0.75,y=1))
fig = dict(data=data, layout=layout)
#url = py.iplot(fig, filename='consumption-vs-price', shared='public')
iplot(fig)

In [61]:
correlation_matrix = df.corr()
correlation = correlation_matrix.ix[0,1]
print('Correlation between monthly variations in gas consumotion and Henry Hub monthly returns: {}%'.format(round(correlation*100,1)))

Correlation between monthly variations in gas consumotion and Henry Hub monthly returns: 22.3%
